### Import Libraries

In [1]:
from bs4 import BeautifulSoup as bs
import requests

### Get the wiki page for Godzilla Franchise

In [2]:
wiki_address = "https://en.wikipedia.org"
godzilla_address = wiki_address + "/wiki/Godzilla_(franchise)"
r = requests.get(godzilla_address)

# Convert to beautiful soup object
soup = bs(r.content)

In [3]:
info_box = soup.find(class_="infobox vevent")
info_rows = info_box.find_all("tr")

In [4]:
def get_subsect(main_row):
    sub_rows = main_row.find_all("li")
    subsect_list = []
    for row in sub_rows:
        row_text = row.find(class_="toctext").get_text()
        if row.find("ul"):
            subsect_list.append(row_text + ":")
            subsect_list.append(get_subsect(row))
        else:
            subsect_list.append(row_text)
    return subsect_list

def get_linked_content(address):
    r_c = requests.get(address)
    soup_c = bs(r_c.content)
    
    toc = soup_c.find(class_="toc").find("ul")
    toc_rows = toc.find_all("li")

    content_list = []
    for row in toc_rows:
        row_text = row.find(class_="toctext").get_text()
        if row_text == 'See also':
            break
        if row.find("ul"):
            content_list.append(row_text + ":")
            content_list.append(get_subsect(row))
            continue

        content_list.append(row_text)

    return content_list

def get_movie_value(row):
    row_text = row.find("td").get_text()
    if "Full list" in row_text:
        content_address = wiki_address + row.find(href = True)['href']
        return get_linked_content(content_address)
    elif row.find("li"):
        return [li.get_text() for li in row.find_all("li")]
    else:
        return row_text

movie_info = {}

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info["Title"] = row.find("th").get_text()
    elif index == 1:
        continue
    else:
        try:
            movie_key = row.find("th").get_text()
            movie_value = get_movie_value(row)
            movie_info[movie_key] = movie_value
        except:
            continue

In [6]:
import json
  
with open("Godzilla.txt", 'w') as f: 
    for key, value in movie_info.items(): 
        f.write('%s: %s\n' % (key, value))